<a href="https://colab.research.google.com/github/alexsc715/advanced_basics/blob/main/ProjetoFinal_AssistenteSa%C3%BAde_v6_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Célula 1: Instalação de Dependências
!pip install -q -U google-generativeai googlemaps

# Célula 2: Importações e Configuração das APIs
import google.generativeai as genai
import googlemaps
import os
import textwrap
from IPython.display import display, Markdown

# --- Configuração da API do Google Gemini ---
# Cole sua API Key do Google Gemini diretamente aqui
GOOGLE_API_KEY = "SUA_GOOGLE_API_KEY_AQUI" # ATENÇÃO: Substitua pela sua chave
genai.configure(api_key=GOOGLE_API_KEY)

# --- Configuração da API do Google Maps ---
# Cole sua API Key do Google Maps diretamente aqui
Maps_API_KEY = "SUA_MAP_API_KEY_AQUI" # ATENÇÃO: Substitua pela sua chave
gmaps = googlemaps.Client(key=Maps_API_KEY)

# Variável para armazenar o estado da conversa entre os agentes
# Usaremos um dicionário para simular variáveis de ambiente no escopo do Colab
estado_conversa = {}

# Funções auxiliares para formatação da saída
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Célula 3: Definição dos Agentes

# Agente 1: Coletor de Sintomas
def agente_coletor_sintomas():
    """
    Pede ao usuário para descrever seus sintomas e armazena a informação.
    """
    print("Olá! Sou o Assistente Saúde. Para começar, por favor, descreva os sintomas que você está sentindo.")
    sintomas_usuario = input("Seus sintomas: ")
    estado_conversa['sintomas_usuario'] = sintomas_usuario
    print("\nObrigado por compartilhar seus sintomas.")
    return True # Indica que o agente concluiu com sucesso

# Agente 2: Pesquisa de Doenças e Especialidades
def agente_pesquisa_doencas_especialidades():
    """
    Usa o Google Gemini para pesquisar possíveis doenças e especialidades médicas
    com base nos sintomas fornecidos.
    """
    print("\n--- Agente de Pesquisa de Doenças e Especialidades ---")
    if 'sintomas_usuario' not in estado_conversa or not estado_conversa['sintomas_usuario']:
        print("Erro: Sintomas não foram informados.")
        return False

    sintomas = estado_conversa['sintomas_usuario']
    print(f"Buscando informações para os sintomas: {sintomas}")

    try:
        model = genai.GenerativeModel('gemini-2.0-flash-001') # Ou outro modelo adequado
        prompt = f"""
        Com base nos seguintes sintomas: '{sintomas}', liste até 3 possíveis condições ou doenças associadas
        e a especialidade médica mais provável para tratar cada uma.
        Formato da resposta para cada condição:
        Condição: [Nome da Condição]
        Especialidade Médica Provável: [Nome da Especialidade]
        ---
        IMPORTANTE: Inclua um aviso de que esta é uma análise preliminar e não substitui uma consulta médica.
        """
        response = model.generate_content(prompt)

        estado_conversa['resultados_pesquisa_doencas'] = response.text
        print("\nResultados da pesquisa preliminar:")
        display(to_markdown(response.text))
        return True
    except Exception as e:
        print(f"Ocorreu um erro ao contatar o Google Gemini: {e}")
        estado_conversa['resultados_pesquisa_doencas'] = "Erro na pesquisa."
        return False

# Agente 3: Determinador da Especialidade Principal
def agente_determinador_especialidade_principal():
    """
    Analisa os resultados da pesquisa anterior e determina a especialidade mais indicada
    para o usuário buscar primeiro.
    """
    print("\n--- Agente Determinador de Especialidade Principal ---")
    if 'resultados_pesquisa_doencas' not in estado_conversa or "Erro na pesquisa" in estado_conversa['resultados_pesquisa_doencas']:
        print("Erro: Resultados da pesquisa de doenças não estão disponíveis ou contêm erro.")
        return False

    resultados_pesquisa = estado_conversa['resultados_pesquisa_doencas']

    try:
        model = genai.GenerativeModel('gemini-2.0-flash-001')
        prompt = f"""
        Com base na seguinte lista de possíveis condições e especialidades médicas:
        '{resultados_pesquisa}'

        Qual é a especialidade médica que parece ser a mais indicada para o usuário procurar primeiro,
        considerando os sintomas e as possíveis condições listadas?
        Responda APENAS com o nome da especialidade médica principal.
        Por exemplo: 'Cardiologia' ou 'Clínica Geral'.
        """
        response = model.generate_content(prompt)
        especialidade_principal = response.text.strip()

        estado_conversa['especialidade_principal_indicada'] = especialidade_principal
        print(f"\nCom base na análise, a especialidade médica mais indicada para você procurar inicialmente é: {especialidade_principal}")
        print("Lembre-se: esta é uma sugestão baseada nas informações fornecidas e não um diagnóstico.")
        return True
    except Exception as e:
        print(f"Ocorreu um erro ao determinar a especialidade principal: {e}")
        estado_conversa['especialidade_principal_indicada'] = "Erro na determinação da especialidade."
        return False

# Agente 4: Consulta sobre Busca Local de Clínicas
def agente_consulta_busca_local():
    """
    Pergunta ao usuário se deseja buscar clínicas ou consultórios próximos
    e coleta a cidade/UF se a resposta for sim.
    """
    print("\n--- Agente de Consulta para Busca Local ---")
    if 'especialidade_principal_indicada' not in estado_conversa or "Erro" in estado_conversa['especialidade_principal_indicada']:
        print("Não foi possível determinar uma especialidade principal para a busca.")
        return False # Indica que não pode prosseguir

    especialidade = estado_conversa['especialidade_principal_indicada']
    resposta = input(f"Você gostaria de buscar clínicas ou consultórios de '{especialidade}' próximos à sua casa? (sim/não): ").strip().lower()

    if resposta == 'sim':
        print("Ótimo! Para prosseguir, por favor, informe:")
        cidade = input("Cidade: ").strip()
        uf = input("UF (Estado, ex: SP, RJ, MG): ").strip().upper()
        estado_conversa['cidade_usuario'] = cidade
        estado_conversa['uf_usuario'] = uf
        estado_conversa['buscar_clinicas'] = True
        print("\nInformações de localização recebidas.")
        return True
    elif resposta == 'não':
        estado_conversa['buscar_clinicas'] = False
        print("\nEntendido. Espero que as informações anteriores tenham sido úteis. Cuide-se e procure um médico para um diagnóstico preciso!")
        return False # Indica que o fluxo principal de busca de clínicas não continuará
    else:
        print("Resposta inválida. Por favor, responda com 'sim' ou 'não'.")
        return agente_consulta_busca_local() # Tenta novamente

# Agente 5: Buscador de Clínicas
def agente_buscador_clinicas():
    """
    Busca no Google Maps por clínicas ou consultórios da especialidade indicada,
    na cidade e UF fornecidas.
    """
    print("\n--- Agente Buscador de Clínicas ---")
    if not estado_conversa.get('buscar_clinicas', False):
        # Se o usuário não quis buscar clínicas, ou se houve erro anterior na especialidade.
        if not estado_conversa.get('especialidade_principal_indicada') or "Erro" in estado_conversa.get('especialidade_principal_indicada', ''):
             print("Não foi possível realizar a busca de clínicas pois a especialidade não foi determinada.")
        return False # Indica que este agente não precisa rodar ou não pode rodar

    especialidade = estado_conversa['especialidade_principal_indicada']
    cidade = estado_conversa['cidade_usuario']
    uf = estado_conversa['uf_usuario']

    query = f"clínicas de {especialidade} em {cidade}, {uf}"
    print(f"Buscando por: {query}")

    try:
        # Primeiro, obtemos as coordenadas da cidade para melhorar a busca (opcional, mas recomendado)
        geocode_result = gmaps.geocode(f'{cidade}, {uf}')
        if not geocode_result:
            print(f"Não foi possível encontrar a localização: {cidade}, {uf}. Tente novamente com dados válidos.")
            return False

        location = geocode_result[0]['geometry']['location']
        lat = location['lat']
        lng = location['lng']

        # Busca por lugares (clínicas/consultórios)
        places_result = gmaps.places(query=query, location=(lat, lng), radius=20000, language='pt-BR') # Raio de 20km, ajuste conforme necessidade

        if places_result and places_result.get('results'):
            print("\nEncontrei os seguintes locais (até 5):")
            count = 0
            for place in places_result['results']:
                if count >= 5:
                    break
                nome = place.get('name')
                endereco = place.get('formatted_address', 'Endereço não disponível')
                place_id = place.get('place_id')
                site = "Não informado"
                map_link = "Não disponível"

                if place_id:
                    # Detalhes do local para pegar o website
                    details = gmaps.place(place_id=place_id, fields=['website', 'url'], language='pt-BR')
                    if details and details.get('result'):
                        site = details['result'].get('website', 'Não informado')
                        map_link = details['result'].get('url', 'Não disponível')

                print(f"\nNome: {nome}")
                print(f"Endereço: {endereco}")
                if site != "Não informado":
                    print(f"Website: {site}")
                if map_link != "Não disponível":
                    print(f"Link no Google Maps: {map_link}")
                print("---")
                count += 1
            if count == 0:
                print("Não encontrei clínicas ou consultórios com os critérios informados nas proximidades.")
        else:
            print("Não foram encontrados resultados para sua busca de clínicas.")

    except Exception as e:
        print(f"Ocorreu um erro durante a busca no Google Maps: {e}")
        return False

    # Pergunta se quer refazer a pesquisa
    resposta_refazer = input("\nGostaria de refazer a pesquisa desde o início? (sim/não): ").strip().lower()
    if resposta_refazer == 'sim':
        estado_conversa['refazer_pesquisa'] = True
    else:
        estado_conversa['refazer_pesquisa'] = False
        print("\nObrigado por utilizar o Assistente Saúde! Lembre-se que este chatbot é uma ferramenta de auxílio e não substitui a avaliação de um profissional de saúde. Cuide-se!")
    return True


# Célula 4: Agente Orquestrador e Loop Principal

def orquestrador():
    """
    Orquestra a execução dos agentes na sequência correta.
    """
    global estado_conversa # Permite modificar o dicionário global
    estado_conversa = {} # Limpa o estado para uma nova interação (se for o caso de loop)

    print("Bem-vindo ao Assistente Saúde!")
    print("Este chatbot tem como objetivo fornecer informações preliminares e não substitui uma consulta médica.")
    print("="*50)

    # Etapa 1: Coletar Sintomas
    if not agente_coletor_sintomas():
        print("Não foi possível coletar os sintomas. Encerrando.")
        return

    # Etapa 2: Pesquisar Doenças e Especialidades
    if not agente_pesquisa_doencas_especialidades():
        print("Não foi possível realizar a pesquisa de doenças. Encerrando.")
        return

    # Etapa 3: Determinar Especialidade Principal
    if not agente_determinador_especialidade_principal():
        print("Não foi possível determinar a especialidade principal. Encerrando.")
        return

    # Etapa 4: Consultar sobre Busca Local
    # Este agente retorna False se o usuário disser 'não' para a busca,
    # mas o fluxo principal não deve ser interrompido por isso, apenas o agente 5 não rodará.
    if not agente_consulta_busca_local():
        if not estado_conversa.get('buscar_clinicas', True): # Se explicitamente disse não
             # A mensagem de encerramento já foi dada pelo agente_consulta_busca_local
            return # Encerra aqui se o usuário não quis buscar clínicas

    # Etapa 5: Buscar Clínicas (só executa se o usuário quis e as etapas anteriores foram bem sucedidas)
    if estado_conversa.get('buscar_clinicas', False): # Verifica se o usuário optou por buscar
        if not agente_buscador_clinicas():
            print("Houve um problema na busca por clínicas.")
            # Mesmo com problema aqui, perguntamos se quer refazer
            if not estado_conversa.get('refazer_pesquisa', False):
                 print("\nObrigado por utilizar o Assistente Saúde! Lembre-se que este chatbot é uma ferramenta de auxílio e não substitui a avaliação de um profissional de saúde. Cuide-se!")

    print("="*50)


# Loop principal para permitir refazer a pesquisa
continuar_chat = True
while continuar_chat:
    orquestrador()
    # Verifica se o usuário quer refazer, decisão tomada no final do agente_buscador_clinicas
    # ou se o fluxo foi interrompido antes e 'refazer_pesquisa' não foi definido.
    if not estado_conversa.get('refazer_pesquisa', False):
        continuar_chat = False
        if 'sintomas_usuario' in estado_conversa : # Se houve alguma interação significativa antes de sair
             if not estado_conversa.get('buscar_clinicas', True) and not estado_conversa.get('refazer_pesquisa', False) and not estado_conversa.get('cidade_usuario'):
                # Caso onde o usuário disse não para buscar clínicas e não quer refazer (mensagem já dada).
                pass
             elif not estado_conversa.get('refazer_pesquisa', False): # Mensagem final geral se não foi dada antes
                print("\nConversa encerrada. Cuide-se e até a próxima!")
    else:
        print("\nReiniciando o Assistente Saúde para uma nova consulta...\n")
        # O estado da conversa será limpo no início da próxima chamada ao orquestrador

Bem-vindo ao Assistente Saúde!
Este chatbot tem como objetivo fornecer informações preliminares e não substitui uma consulta médica.
Olá! Sou o Assistente Saúde. Para começar, por favor, descreva os sintomas que você está sentindo.
Seus sintomas: dor de cabeça, febre, dor de garganta

Obrigado por compartilhar seus sintomas.

--- Agente de Pesquisa de Doenças e Especialidades ---
Buscando informações para os sintomas: dor de cabeça, febre, dor de garganta

Resultados da pesquisa preliminar:


> **Aviso Importante:** Esta é uma análise preliminar baseada nos sintomas fornecidos e não substitui uma consulta médica profissional. É fundamental procurar um médico para um diagnóstico preciso e tratamento adequado.
> 
> Condição: Gripe (Influenza)
> Especialidade Médica Provável: Clínica Médica/Medicina de Família e Comunidade
> 
> ---
> 
> Condição: Amigdalite
> Especialidade Médica Provável: Otorrinolaringologia
> 
> ---
> 
> Condição: COVID-19
> Especialidade Médica Provável: Clínica Médica/Infectologia



--- Agente Determinador de Especialidade Principal ---

Com base na análise, a especialidade médica mais indicada para você procurar inicialmente é: Clínica Médica
Lembre-se: esta é uma sugestão baseada nas informações fornecidas e não um diagnóstico.

--- Agente de Consulta para Busca Local ---
Você gostaria de buscar clínicas ou consultórios de 'Clínica Médica' próximos à sua casa? (sim/não): sim
Ótimo! Para prosseguir, por favor, informe:
Cidade: Brasília
UF (Estado, ex: SP, RJ, MG): DF

Informações de localização recebidas.

--- Agente Buscador de Clínicas ---
Buscando por: clínicas de Clínica Médica em Brasília, DF

Encontrei os seguintes locais (até 5):

Nome: Minha Clínica - Asa Sul
Endereço: SCS, Quadra 02, Bloco B, Lote 20 Edifício Palácio do Comércio, Sobreloja 02 - Asa Sul, Brasília - DF, 70302-000, Brasil
Website: http://www.aminhaclinica.com.br/
Link no Google Maps: https://maps.google.com/?cid=7336555838286342325
---

Nome: Clínica Médica
Endereço: SGAN Q 2 B. A L. 1 - B

KeyboardInterrupt: Interrupted by user